In [1]:
from astropy.io import fits
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd
import redh3p as rhp

from lmfit import Model

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)

In [81]:
line_list_file = 'h3p_line_list_neale_1996_detailed.txt'

header = ['J_upper', '$\Omega$_upper in cm$^{-1}$', '$\Omega$ in cm$^{-1}$', 'A', 'g_ns']
line_list = pd.read_csv(line_list_file, sep='\s+', skiprows=1, header=None)
line_list.columns = header

line_list

,J_upper,$\Omega$_upper in cm$^{-1}$,$\Omega$ in cm$^{-1}$,A,g_ns
0,0,15312.6,457.273,1.081,2.66667
1,1,11342.5,506.225,1.191,2.66667
2,1,14074.5,534.172,1.266,2.66667
3,3,12367.5,548.993,1.087,2.66667
4,0,11260.6,567.597,1.887,2.66667
...,...,...,...,...,...
436153,0,15058.2,12574.100,1.007,2.66667
436154,20,20448.5,12675.500,1.166,2.66667
436155,1,15768.4,13019.700,1.070,2.66667
436156,0,16199.0,13022.400,1.111,2.66667


In [97]:
line_list_to_add = line_list[(line_list['$\Omega$ in cm$^{-1}$'] > 4686) &
                             (line_list['$\Omega$ in cm$^{-1}$'] < 4687)]

line_list_to_add

,J_upper,$\Omega$_upper in cm$^{-1}$,$\Omega$ in cm$^{-1}$,A,g_ns
381228,7,14510.5,4686.03,11.340,2.66667
381229,15,12931.3,4686.03,21.450,2.66667
381230,10,14028.5,4686.04,1.314,2.66667
381231,14,17469.0,4686.05,10.300,2.66667
381232,8,12625.3,4686.08,1.178,2.66667
...,...,...,...,...,...
381313,14,17344.6,4686.98,3.678,2.66667
381314,13,16660.1,4686.98,3.274,2.66667
381315,15,15366.3,4686.99,1.234,2.66667
381316,6,13172.2,4686.99,53.740,2.66667


In [100]:
line_list_subset = pd.read_csv('h3p_line_list_neale_1996_subset.txt', sep='\s+', skiprows=1, header=None)
line_list_subset.columns = header

list1 = line_list_subset[line_list_subset['$\Omega$ in cm$^{-1}$']<4686]
list2 = list1.append(line_list_to_add)
list3 = list2.append(line_list_subset[line_list_subset['$\Omega$ in cm$^{-1}$']>4687])
list3

,J_upper,$\Omega$_upper in cm$^{-1}$,$\Omega$ in cm$^{-1}$,A,g_ns
0,8,8914.37,1490.87,11.95,4.0
1,7,8802.72,1514.11,12.12,2.0
2,7,9075.56,1519.22,11.96,4.0
3,9,7423.50,1542.78,14.56,4.0
4,6,8485.19,1564.11,10.82,4.0
...,...,...,...,...,...
3022,0,7806.53,7806.54,14.49,2.0
3023,1,7925.77,7820.59,14.14,2.0
3024,3,7927.82,7822.64,10.55,2.0
3025,2,8078.34,7827.11,13.54,4.0


In [102]:
list3.to_csv('h3p_line_list_neale_1996_subset_new.txt', sep=' ', header=None, index=None, mode='a')

In [88]:
line_list_subset[line_list_subset['$\Omega$ in cm$^{-1}$']>4687]

,J_upper,$\Omega$_upper in cm$^{-1}$,$\Omega$ in cm$^{-1}$,A,g_ns
2691,6,6865.07,4687.21,14.41,2
2692,8,7399.56,4688.02,54.33,4
2693,2,7687.96,4688.60,12.58,2
2694,4,9729.90,4688.74,127.40,2
2695,3,7953.84,4691.84,165.30,2
...,...,...,...,...,...
3022,0,7806.53,7806.54,14.49,2
3023,1,7925.77,7820.59,14.14,2
3024,3,7927.82,7822.64,10.55,2
3025,2,8078.34,7827.11,13.54,4


In [108]:
list4 = list3[(list3['$\Omega$ in cm$^{-1}$']>4651) & (list3['$\Omega$ in cm$^{-1}$'] < 4740)]
list4

,J_upper,$\Omega$_upper in cm$^{-1}$,$\Omega$ in cm$^{-1}$,A,g_ns
2654,2,9872.07,4653.88,86.38,2.0
2655,6,6330.73,4653.96,10.35,2.0
2656,6,9311.89,4654.22,33.27,4.0
2657,8,9048.27,4655.50,28.41,2.0
2658,6,8769.68,4655.90,10.28,4.0
...,...,...,...,...,...
2738,6,6351.60,4735.93,71.38,2.0
2739,8,6702.73,4736.22,17.52,4.0
2740,1,9615.02,4736.41,178.60,2.0
2741,3,8336.67,4739.71,50.77,2.0


In [109]:
list4.to_csv('h3p_line_list_neale_1996_subset_211_215.txt', sep=' ', header=None, index=None, mode='a')

In [87]:
line_list_subset[line_list_subset['$\Omega$ in cm$^{-1}$']<4686]['$\Omega$ in cm$^{-1}$'].to_numpy()

array([1490.87, 1514.11, 1519.22, ..., 4684.46, 4684.54, 4684.74])

In [68]:
wn = line_list['$\Omega$ in cm$^{-1}$'].to_numpy()
wl = wn[np.where((wn < 1 / (2.1215 * 1e-4)) & (wn > 1 / (2.1445 * 1e-4)))]
wn[np.where((wn > 4686) & (wn < 4687))]

array([4686.03, 4686.03, 4686.04, 4686.05, 4686.08, 4686.1 , 4686.1 ,
       4686.1 , 4686.13, 4686.15, 4686.18, 4686.18, 4686.19, 4686.21,
       4686.21, 4686.22, 4686.22, 4686.26, 4686.28, 4686.28, 4686.29,
       4686.33, 4686.33, 4686.34, 4686.34, 4686.35, 4686.35, 4686.42,
       4686.43, 4686.43, 4686.43, 4686.46, 4686.47, 4686.48, 4686.49,
       4686.5 , 4686.51, 4686.51, 4686.52, 4686.53, 4686.54, 4686.55,
       4686.56, 4686.57, 4686.58, 4686.6 , 4686.61, 4686.63, 4686.63,
       4686.63, 4686.63, 4686.64, 4686.65, 4686.66, 4686.69, 4686.69,
       4686.73, 4686.74, 4686.74, 4686.75, 4686.76, 4686.76, 4686.77,
       4686.79, 4686.79, 4686.8 , 4686.81, 4686.81, 4686.82, 4686.83,
       4686.83, 4686.84, 4686.84, 4686.85, 4686.85, 4686.85, 4686.86,
       4686.91, 4686.92, 4686.92, 4686.93, 4686.94, 4686.95, 4686.97,
       4686.97, 4686.98, 4686.98, 4686.99, 4686.99, 4686.99])

In [54]:
wl.shape

(4358,)

In [31]:
line_list.loc[436156]

J_upper                            0.00000
$\Omega$_upper in cm$^{-1}$    16199.00000
$\Omega$ in cm$^{-1}$          13022.40000
A                                  1.11100
g$_{ns}$                           2.66667
Name: 436156, dtype: float64

In [103]:
1 / (2.11 * 1e-4),  1 / (2.15 * 1e-4)

(4739.336492890995, 4651.162790697675)

In [63]:
1 / (2.1337 * 1e-4)

4686.6944743872145